In [1]:
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.mixture import GaussianMixture
from scipy.special import kl_div
import MDAnalysis as mda
import sys

## Making test data

In [10]:

cosmos1 = mda.Universe("../binding_spots_project/gpcr_sampling/b2ar/chol/site-3/step6.0_minimization.gro",
                      "../binding_spots_project/gpcr_sampling/b2ar/chol/site-3/short10.xtc")
cosmos2 = mda.Universe("../binding_spots_project/gpcr_sampling/b2ar/sdpc/short.gro",
                      "../binding_spots_project/gpcr_sampling/b2ar/sdpc/short10.xtc")

selection = "name CA"

pos1 = np.zeros(3).reshape(1,3)

for ts in cosmos1.trajectory[0:-1:5]:
    pos1 = np.concatenate((pos1, cosmos1.select_atoms(selection).positions))

pos1 = pos1[~np.all(pos1 == 0, axis=1)]    
pos1 = np.concatenate((pos1, np.ones(pos1.shape[0]).reshape(pos1.shape[0], 1)), axis=1)


pos2 = np.zeros(3).reshape(1,3)

for ts in cosmos2.trajectory[0:-1:5]:
    pos2 = np.concatenate((pos2, cosmos2.select_atoms(selection).positions))

pos2 = pos2[~np.all(pos2 == 0, axis=1)]    
pos2 = np.concatenate((pos2, np.ones(pos2.shape[0]).reshape(pos2.shape[0], 1)), axis=1)

X = np.concatenate((pos1, pos2))


## FLOW

In [19]:

default_reducers = [PCA()]
default_classifiers = [GaussianMixture()]

default_settings = {
    "n_components": [2,3,4,5,6]
}


test_params = {
    PCA.__name__: default_settings,
    GaussianMixture.__name__: default_settings
}

test_steps = [(PCA.__name__, PCA()), (GaussianMixture.__name__, GaussianMixture())]

In [26]:

def build_pipeline(steps, params):
    pipeline = Pipeline(steps=steps)
    for k, v in params.items():
        for kk, vv in v.items():
            pipeline.set_params(**{k + "__" + kk: vv})
    return pipeline




